In [1]:
import os
os.chdir("../")
print(os.getcwd())

c:\Users\Amrit\Intilt\Intrade_ai


In [2]:
import pandas as pd
import numpy as np
import datetime
from utils.data_cleaning import get_continuous_1min_data

In [3]:
def convert_timeframe_min(df, timeframe):
    df.index = pd.to_datetime(df.index)
    df['day'] = df.index.normalize()
    gp = df.groupby('day')
    dfList = []
    for k, res in gp:
        resampledf = res.resample(timeframe, origin='start').agg({'open':'first',
                                                        'high':'max',
                                                        'low':'min',
                                                        'close':'last'})
        resampledf.reset_index(inplace=True)
        dfList.append(resampledf)
    finaldf = pd.concat(dfList, ignore_index=True)
    finaldf = finaldf.set_index('datetime')
    return finaldf

In [23]:
data1 = pd.read_csv('data/Nifty/nifty_1min.csv')

In [7]:
nifty_min_continous = get_continuous_1min_data("Nifty",data1)

In [26]:
df1 = nifty_min_continous.copy()

In [27]:
df1.isnull().sum()

open     792
high     792
low      792
close    792
dtype: int64

In [31]:
df1.to_csv('1min_noise.csv')

In [30]:
# Impute missing values using linear interpolation
df1.interpolate(method='linear', inplace=True)

# Add noise to the imputed values
noise = np.random.normal(0, 0.02, df1.shape)
df1 += noise

In [12]:
df1

,open,high,low,close
datetime,,,,
2011-01-03 09:15:00,6178.265448,6178.545091,6171.751749,6172.049639
2011-01-03 09:16:00,6173.780831,6173.778451,6169.033969,6170.864577
2011-01-03 09:17:00,6171.503789,6174.645863,6170.912416,6174.645227
2011-01-03 09:18:00,6174.004451,6178.190445,6174.028361,6177.694238
2011-01-03 09:19:00,6177.799857,6178.245541,6176.759080,6177.035301
...,...,...,...,...
2021-12-31 15:25:00,17357.862545,17358.515126,17353.779614,17356.308360
2021-12-31 15:26:00,17356.107626,17359.724604,17355.573548,17359.311384
2021-12-31 15:27:00,17358.806711,17360.484380,17356.182261,17359.849127


In [21]:
df1.loc['2017-07-10 09:20:00':'2017-07-10 09:35:00']

,open,high,low,close,day
datetime,,,,,
2017-07-10 09:20:00,9686.514168,9761.637982,9685.083369,9685.685289,2017-07-10
2017-07-10 09:21:00,9685.742057,9687.412061,9684.742712,9686.609939,2017-07-10
2017-07-10 09:22:00,9686.043705,9732.205045,9685.350536,9688.189467,2017-07-10
2017-07-10 09:23:00,9688.796057,9688.845650,9685.543348,9685.850214,2017-07-10
2017-07-10 09:24:00,9686.185305,9686.237435,9684.053667,9684.037096,2017-07-10
2017-07-10 09:25:00,9702.750720,9703.375554,9687.992932,9691.833412,2017-07-10
2017-07-10 09:26:00,9719.338147,9720.584166,9691.975341,9699.637166,2017-07-10
2017-07-10 09:27:00,9735.916402,9737.768267,9695.842226,9707.464524,2017-07-10
2017-07-10 09:28:00,9752.432245,9754.979041,9699.786808,9715.241980,2017-07-10


In [13]:
timeframe = '15T'
data_15_min = convert_timeframe_min(df1, timeframe)

In [14]:
data_15_min.isnull().sum()

open     14
high     14
low      14
close    14
dtype: int64

In [15]:
data_15_min[data_15_min.isna().any(axis=1)]

,open,high,low,close
datetime,,,,
2017-07-10 09:30:00,NaN,NaN,NaN,NaN
2017-07-10 09:45:00,NaN,NaN,NaN,NaN
2017-07-10 10:00:00,NaN,NaN,NaN,NaN
2017-07-10 10:15:00,NaN,NaN,NaN,NaN
2017-07-10 10:30:00,NaN,NaN,NaN,NaN
2017-07-10 10:45:00,NaN,NaN,NaN,NaN
2017-07-10 11:00:00,NaN,NaN,NaN,NaN
2017-07-10 11:15:00,NaN,NaN,NaN,NaN
2017-07-10 11:30:00,NaN,NaN,NaN,NaN


In [25]:
data_15_min.loc['2021-02-24 10:00:00':'2021-02-24 12:30:00']

,open,high,low,close
datetime,,,,
2021-02-24 10:00:00,14780.458033,14827.856354,14776.316876,14827.876305
2021-02-24 10:15:00,14827.937289,14843.730714,14827.533543,14843.704274
2021-02-24 10:30:00,14841.909968,14859.607545,14840.623161,14859.525806
2021-02-24 10:45:00,14855.875100,14875.462789,14853.851243,14875.284579
2021-02-24 11:00:00,14869.826969,14891.302182,14866.986954,14891.065507
2021-02-24 11:15:00,14883.803061,14901.852339,14880.146303,14901.655495
2021-02-24 11:30:00,NaN,NaN,NaN,NaN
2021-02-24 11:45:00,NaN,NaN,NaN,NaN
2021-02-24 12:00:00,14893.094269,14908.185233,14888.903469,14907.971532
